In [ ]:
import pandas as pd
from google.colab import files

# Upload the file
uploaded = files.upload()

# Get the filename (in case you upload a different file)
filename = next(iter(uploaded))

# Load dataset
df = pd.read_csv(filename)
df.head()

import pandas as pd
from google.colab import files

files.upload()  # Upload player_stats_2023.csv
df = pd.read_csv('player_stats_2023.csv')
df.head()

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv('player_stats_2023.csv')
df.head()


In [ ]:
# Drop missing or invalid data
df.dropna(subset=['minutes_played', 'goals'], inplace=True)

# Normalize stats per 90 minutes
df['goals_per_90'] = df['goals'] / df['minutes_played'] * 90
df['assists_per_90'] = df['assists'] / df['minutes_played'] * 90
df['tackles_per_90'] = df['tackles'] / df['minutes_played'] * 90


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Compare positions
sns.boxplot(x='position', y='goals_per_90', data=df)
plt.title('Goals per 90 by Position')
plt.show()

# Correlation heatmap
sns.heatmap(df[['goals_per_90', 'assists_per_90', 'tackles_per_90', 'market_value']].corr(), annot=True)
plt.title('Feature Correlations')
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Define features and target
features = ['goals_per_90', 'assists_per_90', 'tackles_per_90']
X = df[features]
y = df['market_value']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Evaluate
print("R² Score:", model.score(X_test, y_test))


In [ ]:
import shap

# Explain model predictions
explainer = shap.Explainer(model, X_test)
shap_values = explainer(X_test)

# Visualize SHAP
shap.plots.beeswarm(shap_values)


In [ ]:
# Define total contribution metric
df['contribution_per_90'] = df['goals_per_90'] + df['assists_per_90']

# Select top forwards
top_forwards = df[df['position'] == 'Forward'].sort_values(by='contribution_per_90', ascending=False).head(10)

# Display
print(top_forwards[['player_name', 'contribution_per_90', 'market_value']])


In [ ]:
top_forwards.to_csv('top_10_forwards.csv', index=False)
